# Aqui es la construcción del RAG

In [49]:
from elasticsearch import Elasticsearch
from openai import OpenAI
import json

In [50]:
#preparing json
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [14]:
#preparing elasticsearch
es_client = Elasticsearch('http://localhost:9200') 
es_client.info()

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "Título": {"type": "keyword"},
            "Artículo": {"type": "text"} 
        }
    }
}

index_name = "transito-v1"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

for doc in docs_raw:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [ ]:
#preparing openai
client = OpenAI()

In [21]:

def elastic_search(query_string):
    query_const={
        "match": {
            "Artículo":query_string
        }
    }
    response = es_client.search(index=index_name,query=query_const)

    results_docs = []

    for hit in response['hits']['hits']:
        results_docs.append(hit['_source'])

    return results_docs

def build_prompt(query, search_results):
    prompt_template = """
You're an assistant that knows the mexican traffic regulations, more specific the Ciudad de México traffic regulations. 
Answer the QUESTION based on the CONTEXT from the database, also the answer is need it to be in spanish.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
         context = context + f"Título: {doc['Título']}\nArtículo: {doc['Artículo']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [22]:
respuesta = rag("Qué dice el artículo 33")

In [24]:
print(respuesta)

El artículo 33 de las regulaciones de tránsito en la Ciudad de México establece que, aun cuando el conductor o alguna otra persona esté presente, los vehículos motorizados estacionados podrán ser inmovilizados por un agente autorizado en los siguientes casos:

I. Cuando los vehículos se encuentren estacionados en lugares prohibidos, de acuerdo con el artículo 30 en sus fracciones IV, VII, XIII y XVIII.

II. Para vehículos con placas foráneas estacionados en zonas con sistemas de cobro por estacionamiento en vía pública, en los siguientes casos:
   a) No se haya cubierto la cuota de estacionamiento en el momento de la revisión.
   b) Haya concluido el tiempo pagado.
   c) Si hay un comprobante de pago, éste no sea visible desde el exterior del vehículo, o el número de placas no coincida, o la fecha del comprobante sea distinta.
   d) El vehículo esté fuera de un cajón y/o zona marcada para el estacionamiento, o esté obstruyendo otro cajón o acceso a cochera, exceptuando aquellos vehícul

In [1]:
import hashlib

In [54]:
def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['Título']}-{doc['Artículo'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [61]:
for doc in docs_raw:
    doc['id'] = generate_document_id(doc)

In [70]:
import json

In [75]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(docs_raw, f_out, indent=2, ensure_ascii=False)

In [76]:

!head documents-with-ids.json

[
  {
    "Título": "Título 1",
    "Artículo": "Artículo 1.- El presente reglamento tiene por objeto regular la circulación de peatones y vehículos en la vía \npública y la seguridad vial en la Ciudad de México.  \n \nLas disposiciones de este reglamento son aplicables a peatones, ciclistas, conductores, pasajeros y \npropietarios de cualquier tipo de vehículo matriculado en el país o el extranjero y que circule en el territorio \nde la Ciudad de México. En el presente ordenamiento se establecen las normas respecto a sus movimientos \ny estacionamiento, en observancia a lo establecido en las leyes, reglamentos, acuerdos, decretos y \nnormatividad local vigente, así como las maniobras de ascenso y descenso de pasajeros o de carga y \ndescarga. De igual forma, determina las condiciones legales y de seguridad a las que se deben ajustar los \nvehículos y sus conductores para su circulación, y establece los esquemas de sanciones por la comisión de \ninfracciones a los preceptos contenidos 

In [ ]:
prompt_template = """
You emulate a person that have questions that are related with the Ciudad de México traffic regulations.
Formulate 5 questions ths person might ask based on the Ciudad de México traffic regulations. 
Should contain the answer to the questions, and the questions should be complete and not too short, also 
the queastions and answers need to be in spanish.

The record:

Título: {Título}
Pregunta: {Pregunta}
Respuesta: {}

You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()